In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV

from data_reader import read_data
from data_preprocessing import preprocess_data
from feature_extractor import extract_features
from random_forest_classification import train, predict

%matplotlib inline

In [ ]:
data = read_data().head(500000)
data = preprocess_data(data)
features = extract_features(data)

In [ ]:
rf = train(train_data, result_column)
predicted = predict(rf, test_data)